In [1]:
%use adventOfCode

In [2]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 8)

In [3]:
aoc.viewPartOne()

--- Day 8: Playground --- 
 Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad. 
 You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground! 
 Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes . 
 Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes. 
 The Elves are trying to figure out which junction boxes to connect so that electricity can reach every junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input). 
 For example: 
 162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
 
 This list describes the position of 20 junction boxes, one per line. Each position is given as X,Y,Z coordinates. So, the first junction box in the list is at X=162 , Y=817 , Z=812 . 
 To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance . In this example, the two junction boxes which are closest together are 162,817,812 and 425,690,689 . 
 By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit . After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits. 
 Now, the two junction boxes which are closest together but aren't already directly connected are 162,817,812 and 431,825,988 . After connecting them, since 162,817,812 is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each. 
 The next two junction boxes to connect are 906,360,560 and 805,96,715 . After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each. 
 The next two junction boxes are 431,825,988 and 425,690,689 . Because these two junction boxes were already in the same circuit , nothing happens! 
 This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be. 
 After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40 . 
 Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?

In [44]:
val testInput = """
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689""".trim()

In [6]:
data class Vec(val x: Int, val y: Int, val z: Int)

In [35]:
fun List<String>.parseInput() = map { line ->
    val (x, y, z) = line.split(",")
    Vec(x.toInt(), y.toInt(), z.toInt())
}


In [63]:
fun Int.pow(exp: Int) = toDouble().pow(exp)

fun Vec.distance(other: Vec) = Math.sqrt((x - other.x).pow(2) + (y - other.y).pow(2) + (z - other.z).pow(2))

fun part1(input: String): Int {

    val boxes = input.lines().parseInput()
    val possibleConnections = buildMap {

        for (i in boxes.indices) {
            for (j in i + 1..boxes.lastIndex) {
                put(setOf(boxes[i], boxes[j]), boxes[i].distance(boxes[j]))
            }
        }
    }

    data class State(
        val circuits: List<Set<Vec>>,
        val possibleConnections: Map<Set<Vec>, Double>,
        val lastPair: Set<Vec>,
    )

    val sequence = generateSequence(
        State(
            boxes.map { setOf(it) },
            possibleConnections,
            emptySet()
        )
    ) { (circuits, possibleConnections, _) ->
        val shortestConnection = possibleConnections.minBy { it.value }

        val nearestBoxes = shortestConnection.key.toList()

        val firstCircuit = circuits.find { nearestBoxes.toList()[0] in it }!!
        val secondCircuit = circuits.find { nearestBoxes.toList()[1] in it }!!

        val list = circuits.minusElement(firstCircuit).minusElement(secondCircuit)
        val newCircuits = list.plusElement(buildSet { addAll(firstCircuit); addAll(secondCircuit) })
        val newPossibleConnections = possibleConnections.toMutableMap().apply {
            remove(shortestConnection.key)
        }

        println(shortestConnection.key)
        State(newCircuits, newPossibleConnections, shortestConnection.key)
    }
    val last = sequence.take(11).last()
    return last.circuits.map { it.size }.sortedDescending().take(3).reduce(Int::times)
}

part1(testInput)

[Vec(x=162, y=817, z=812), Vec(x=425, y=690, z=689)]
[Vec(x=162, y=817, z=812), Vec(x=431, y=825, z=988)]
[Vec(x=906, y=360, z=560), Vec(x=805, y=96, z=715)]
[Vec(x=431, y=825, z=988), Vec(x=425, y=690, z=689)]
[Vec(x=862, y=61, z=35), Vec(x=984, y=92, z=344)]
[Vec(x=52, y=470, z=668), Vec(x=117, y=168, z=530)]
[Vec(x=819, y=987, z=18), Vec(x=941, y=993, z=340)]
[Vec(x=906, y=360, z=560), Vec(x=739, y=650, z=466)]
[Vec(x=346, y=949, z=466), Vec(x=425, y=690, z=689)]
[Vec(x=906, y=360, z=560), Vec(x=984, y=92, z=344)]


40

In [121]:
val boxes = aoc.input().lines().parseInput()
solve(boxes, fastMode = false)
    .take(1000).last().circuits.map { it.size }.sortedDescending().take(3).reduce(Int::times)

123420

In [ ]:
aoc.submitPartOne(123420)

--- Part Two --- 
 The Elves were right; they definitely don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in one large circuit . 
 Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at 216,146,977 and 117,168,530 . The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes ( 216 and 117 ) produces 25272 . 
 Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit . What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?

In [115]:
data class State(
    val circuits: Set<Set<Vec>>,
    val possibleConnections: Map<Set<Vec>, Double>,
    val lastPair: Set<Vec>,
)

fun allConnections(boxes: List<Vec>) = buildMap {
    for (i in boxes.indices) {
        for (j in i + 1..boxes.lastIndex) {
            put(
                setOf(boxes[i], boxes[j]),
                boxes[i].distance(boxes[j])
            )
        }
    }
}.toMutableMap()

fun solve(boxes: List<Vec>, fastMode: Boolean = true) = sequence {
    val circuits = boxes.map { setOf(it) }.toMutableSet()

    val possibleConnections = allConnections(boxes)

    while (true) {
        val shortestConnection = possibleConnections.minBy { it.value }

        val nearest = shortestConnection.key
        val firstCircuit = circuits.find { nearest.toList().toList()[0] in it }!!
        val secondCircuit = circuits.find { nearest.toList().toList()[1] in it }!!

        val newCircuit = buildSet { addAll(firstCircuit); addAll(secondCircuit) }
        circuits.apply {
            remove(firstCircuit)
            remove(secondCircuit)
            add(newCircuit)
        }

        possibleConnections.apply {
            val newAsList = newCircuit.toList()

            if (fastMode) {
                for (i in newAsList.indices) {
                    for (j in i + 1..newAsList.lastIndex) {
                        remove(setOf(newAsList[i], newAsList[j]))
                    }
                }
            } else {
                remove(nearest)
            }
        }

        yield(State(circuits as Set<Set<Vec>>, possibleConnections, nearest))
    }
}

In [113]:
fun part2(input: String): Int {
    val lastStep = input.lines().parseInput().solve()
        .first { it.circuits.size == 1 }
    println(lastStep.lastPair)
    val (first, second) = lastStep.lastPair.toList()
    return first.x * second.x
}


In [114]:
val result = part2(aoc.input()).also { DISPLAY(it) }
aoc.submitPartTwo(result)

673096646

[Vec(x=26426, y=1736, z=85661), Vec(x=25471, y=15753, z=85568)]


The answer is correct! Your answer: 673096646 You already completed this part.